In [1]:
using BenchmarkTools
using Distributed
using Base.Threads

using NBInclude
@nbinclude("solve_model.ipynb")


solve_tree (generic function with 5 methods)

Teste simples: gerar draws da uniforme

In [2]:
@btime rand(1000);

  935.294 ns (1 allocation: 7.94 KiB)


In [3]:
@btime for i in 1:1000
    rand()
end

  2.833 μs (0 allocations: 0 bytes)


Usando Threads

In [4]:
#mudei a variavel ambiente JULIA_NUM_THREADS no windows 10
Threads.nthreads()

4

In [6]:
@btime @threads for i in 1:1000
    rand()
    end


  5.620 μs (21 allocations: 3.42 KiB)


### Exemplo de rodar o código da solução do modelo

In [7]:
#Vh/D, L/D, Dj/D
data = [1.0, 0.25, 0.68]; 

#ρ, β, c0, λj
game_parameters = [0.884, 9.84, 0.044, 0.346];

Sem paralelizar

In [8]:
@time for i in 1:2
    solve_tree(data, game_parameters);
end
    

308.328711 seconds (635.93 M allocations: 210.131 GiB, 8.80% gc time)
356.105231 seconds (632.36 M allocations: 209.969 GiB, 9.76% gc time)
666.623594 seconds (1.27 G allocations: 420.816 GiB, 9.30% gc time)


Com paralelização

In [9]:
@time @threads for i in 1:2
    solve_tree(data, game_parameters);
end

558.320433 seconds (1.25 G allocations: 418.808 GiB, 21.37% gc time)
563.971564 seconds (1.26 G allocations: 419.918 GiB, 21.22% gc time)
564.537617 seconds (1.27 G allocations: 420.445 GiB, 21.23% gc time)


In [10]:
@time @threads for i in 1:4
    solve_tree(data, game_parameters);
end

1106.335033 seconds (2.51 G allocations: 838.457 GiB, 32.48% gc time)
1110.202050 seconds (2.53 G allocations: 839.586 GiB, 32.42% gc time)
1110.419510 seconds (2.53 G allocations: 839.661 GiB, 32.42% gc time)
1111.147823 seconds (2.53 G allocations: 839.873 GiB, 32.40% gc time)
1111.836986 seconds (2.53 G allocations: 840.869 GiB, 32.38% gc time)
